## Introduction

System information (for reproducibility):

In [82]:
versioninfo()

Julia Version 1.11.4
Commit 8561cc3d68d (2025-03-10 11:36 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [83]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/slides/02-langs`


Status `~/Documents/github.com/ucla-biostat-257/2025spring/slides/02-langs/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
  [31c24e10] Distributions v0.25.118
  [bdcacae8] LoopVectorization v0.12.172
  [438e738f] PyCall v1.96.4
  [6f49c342] RCall v0.14.6
  [10745b16] Statistics v1.11.1
  [8f399da3] Libdl v1.11.0
  [9a3f8284] Random v1.11.0


In this lecture, we review some popular computer languages commonly used in computational statistics. We want to understand why some languages are faster than others and what are the remedies for the slow languages.

### Poll: Multiple Answer

Which languages have you programmed before?  
A. C or C++  
B. Fortran  
C. Julia  
D. Matlab  
E. Python  

## Types of computer languages

* **Compiled languages** (low-level languages): C/C++, Fortran, ... 
  - Directly compiled to machine code that is executed by CPU 
  - Pros: fast, memory efficient
  - Cons: longer development time, hard to debug

* **Interpreted language** (high-level languages): R, Matlab, Python, SAS IML, JavaScript, ... 
  - Interpreted by interpreter
  - Pros: fast prototyping
  - Cons: excruciatingly slow for loops

* Mixed (dynamic) languages: Matlab (JIT), R (`compiler` package), Julia, Cython, JAVA, ...
  - Pros and cons: between the compiled and interpreted languages

* Script languages: Linux shell scripts, Perl, ...
  - Extremely useful for some data preprocessing and manipulation

* Database languages: SQL, Hive (Hadoop).  
  - Data analysis *never* happens if we do not know how to retrieve data from databases  
  
  [203B (Introduction to Data Science)](https://ucla-biostat-203b.github.io/2025winter/schedule/schedule-lec1.html) covers some basics on scripting and database.

## How high-level languages work?

Comics: [The Life of a Bytecode Language](https://betterprogramming.pub/the-life-of-a-bytecode-language-fca666928e7b)

- Typical execution of a high-level language such as R, Python, and Matlab.

<img src="./r-bytecode.png" align="center" width="400"/>

- To improve efficiency of interpreted languages such as R or Matlab, conventional wisdom is to avoid loops as much as possible. Aka, **vectorize code**
> The only loop you are allowed to have is that for an iterative algorithm.

- When looping is unavoidable, need to code in C, C++, or Fortran. This creates the notorious **two language problem**.  
Success stories: the popular `glmnet` package in R is coded in Fortran; `tidyverse` and `data.table` packages use a lot of RCpp/C++.

<img src="./two_language_problem.jpg" align="center" width="600"/>

- High-level languages have made many efforts to bring themselves closer to the performance of low-level languages such as C, C++, or Fortran, with a variety levels of success.   
    - Matlab has employed JIT (just-in-time compilation) technology since 2002.  
    - Since R 3.4.0 (Apr 2017), the JIT bytecode compiler is enabled by default at its level 3.   
    - Cython is a compiler system based on Python.  

- Modern languages such as Julia tries to solve the two language problem. That is to achieve efficiency without vectorizing code.

- Julia execution.

<img src="./julia_compile.png" align="center" width="400"/>

<img src="./julia_introspect.png" align="center" width="400"/>

## Gibbs sampler example by Doug Bates

- Doug Bates (member of R-Core, author of popular R packages `Matrix`, `lme4`, `RcppEigen`, etc)
    
    > As some of you may know, I have had a (rather late) mid-life crisis and run off with another language called Julia.   
    >
    > -- <cite>Doug Bates (on the `knitr` Google Group)</cite>

- An example from Dr. Doug Bates's slides [Julia for R Programmers](http://www.stat.wisc.edu/~bates/JuliaForRProgrammers.pdf).

- The task is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 \exp(- x y^2 - y^2 + 2y - 4x), \quad x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

### Poll: Multiple choice

Which language will be faster for this Gibbs sampler: R or Julia?  
A. R is much faster than Julia  
B. R is moderately faster than Julia  
C. R is about same speed as Julia  
D. R is moderately slower than Julia  
E. R is much slower than Julia  

### R solution

* The `RCall.jl` package allows us to execute R code without leaving the `Julia` environment. We first define an R function `Rgibbs()`.

In [84]:
using RCall

# show R information
R"""
sessionInfo()
"""

RObject{VecSxp}
R version 4.4.2 (2024-10-31)
Platform: aarch64-apple-darwin20
Running under: macOS Sequoia 15.3.2

Matrix products: default
BLAS:   /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] Rcpp_1.0.14     lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1  
 [5] dplyr_1.1.4     purrr_1.0.2     readr_2.1.5     tidyr_1.3.1    
 [9] tibble_3.2.1    ggplot2_3.5.1   tidyverse_2.0.0 bench_1.1.4    
[13] Matrix_1.7-1   

loaded via a namespace (and not attached):
 [1] gtable_0.3.5     compiler_4.4.2   tidyselect_1.2.1 scales_1.3.0    
 [5] lattice_0.22-6   R6_2.5.1         generics_0.1.3   munsell_0.5.1   
 [9] pillar

In [85]:
# define a function for Gibbs sampling

R"""
library(Matrix)

Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
"""

RObject{ClosSxp}
function (N, thin) 
{
    mat <- matrix(0, nrow = N, ncol = 2)
    x <- y <- 0
    for (i in 1:N) {
        for (j in 1:thin) {
            x <- rgamma(1, 3, y * y + 4)
            y <- rnorm(1, 1/(x + 1), 1/sqrt(2 * (x + 1)))
        }
        mat[i, ] <- c(x, y)
    }
    mat
}


To generate a sample of size 10,000 with a thinning of 500. How long does it take?

In [86]:
R"""
system.time(Rgibbs(10000, 500))
"""

RObject{RealSxp}
   user  system elapsed 
  6.179   0.087   6.289 


### Julia solution

* This is a Julia function for the same Gibbs sampler:

In [87]:
using Distributions

function jgibbs(N, thin)
    mat = zeros(N, 2)
    x = y = 0.0
    for i in 1:N
        for j in 1:thin
            x = rand(Gamma(3, 1 / (y * y + 4)))
            y = rand(Normal(1 / (x + 1), 1 / sqrt(2(x + 1))))
        end
        mat[i, 1] = x
        mat[i, 2] = y
    end
    mat
end

jgibbs (generic function with 1 method)

Generate the same number of samples. How long does it take?

In [88]:
jgibbs(100, 5); # warm-up
@elapsed jgibbs(10000, 500)

0.110164084

We see 40-80 fold speed up of `Julia` over `R` on this example, **with similar coding effort**!

## Comparing C, C++, R, Python, and Julia

To better understand how these languages work, we consider a simple task: summing a vector.

Let's first generate data: 1 million double precision numbers from uniform [0, 1].

In [89]:
using Random

Random.seed!(257) # seed
x = rand(1_000_000) # 1 million random numbers in [0, 1)
sum(x)

500156.3977288406

In this class, we extensively use package `BenchmarkTools.jl` for robust benchmarking. It's the analog of the `microbenchmark` or `bench` package in R.

In [90]:
using BenchmarkTools

### C

We would use the low-level C code as the baseline for copmarison. In Julia, we can easily run compiled C code using the `ccall` function.

In [91]:
using Libdl

C_code = """
#include <stddef.h>

double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -std=c99 -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X :: Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

clang: warning: argument unused during compilation: '-msse3' [-Wunused-command-line-argument]


c_sum (generic function with 1 method)

In [92]:
# make sure it gives same answer
c_sum(x)

500156.39772885485

In [93]:
# dollar sign to interpolate array x into local scope for benchmarking
bm = @benchmark c_sum($x)

BenchmarkTools.Trial: 5954 samples with 1 evaluation per sample.
 Range (min … max):  811.625 μs …  1.383 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     826.958 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   839.000 μs ± 36.869 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▅▅▅▅▅▅▄▄▃▃▂▃▃▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁                               ▁
  ████████████████████████████████████▇▇▇▇█▇▇▅▆▅▅▅▅▅▄▅▃▅▂▅▄▄▄▂ █
  812 μs        Histogram: log(frequency) by time       978 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [94]:
# a dictionary to store median runtime (in milliseconds)
benchmark_result = Dict() 

# store median runtime (in milliseconds)
benchmark_result["C"] = median(bm.times) / 1e6

0.826958

### R, buildin sum

Next we compare to the build in `sum` function in R, which is implemented using C.

In [95]:
R"""
library(bench)
library(tidyverse)

# interpolate x into R workspace
y <- $x
rbm_builtin <- bench::mark(sum(y)) %>%
  print(width = Inf)
""";

# A tibble: 1 x 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 sum(y)       1.33ms   1.37ms      683.        0B        0   342     0
  total_time result    memory             time             gc                
    <bch:tm> <list>    <list>             <list>           <list>            
1      500ms <dbl [1]> <Rprofmem [0 x 3]> <bench_tm [342]> <tibble [342 x 3]>


In [96]:
# store median runtime (in milliseconds)
@rget rbm_builtin # dataframe
benchmark_result["R builtin"] = median(rbm_builtin[!, :median]) * 1000

1.3692360371351242

### R, handwritten loop

Handwritten loop in R is much slower.

In [97]:
R"""
sum_r <- function(x) {
  s <- 0
  for (xi in x) {
    s <- s + xi
  }
  s
}
library(bench)
y <- $x
rbm_loop <- bench::mark(sum_r(y)) %>%
  print(width = Inf)
""";

# A tibble: 1 x 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 sum_r(y)     9.43ms   9.92ms      100.        0B        0    51     0
  total_time result    memory             time            gc               
    <bch:tm> <list>    <list>             <list>          <list>           
1      507ms <dbl [1]> <Rprofmem [0 x 3]> <bench_tm [51]> <tibble [51 x 3]>


In [98]:
# store median runtime (in milliseconds)
@rget rbm_loop # dataframe
benchmark_result["R loop"] = median(rbm_loop[!, :median]) * 1000

9.919499047100544

### R, Rcpp

Rcpp package provides an easy way to incorporate C++ code in R.

In [99]:
R"""
library(Rcpp)

cppFunction('double rcpp_sum(NumericVector x) {
  int n = x.size();
  double total = 0;
  for(int i = 0; i < n; ++i) {
    total += x[i];
  }
  return total;
}')

rcpp_sum
"""

RObject{ClosSxp}
function (x) 
.Call(<pointer: 0x177a78300>, x)


In [100]:
R"""
rbm_rcpp <- bench::mark(rcpp_sum(y)) %>%
  print(width = Inf)
""";

# A tibble: 1 x 13
  expression       min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr>  <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 rcpp_sum(y)    799us    831us     1139.        0B        0   570     0
  total_time result    memory             time             gc                
    <bch:tm> <list>    <list>             <list>           <list>            
1      500ms <dbl [1]> <Rprofmem [0 x 3]> <bench_tm [570]> <tibble [570 x 3]>


In [101]:
# store median runtime (in milliseconds)
@rget rbm_rcpp # dataframe
benchmark_result["R Rcpp"] = median(rbm_rcpp[!, :median]) * 1000

0.8309880213346332

### Python, builtin sum

Built in function `sum` in Python.

In [102]:
using PyCall

PyCall.pyversion

v"3.10.12"

In [103]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
bm = @benchmark $pysum($x)

BenchmarkTools.Trial: 129 samples with 1 evaluation per sample.
 Range (min … max):  37.664 ms … 56.870 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     38.529 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.767 ms ±  1.786 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     █▄  ▅                                                     
  ▄▅▆███▆█▇█▅▆▃▆▃▄▄▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  37.7 ms         Histogram: frequency by time          45 ms <

 Memory estimate: 208 bytes, allocs estimate: 8.

In [104]:
# store median runtime (in miliseconds)
benchmark_result["Python builtin"] = median(bm.times) / 1e6

38.529208

### Python, handwritten loop

In [105]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

bm = @benchmark $sum_py($x)

BenchmarkTools.Trial: 105 samples with 1 evaluation per sample.
 Range (min … max):  46.301 ms … 72.954 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.619 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.856 ms ±  2.558 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

            █▂    ▅▂▅▂ ▂ ▂   ▂▂  ▅ ▂  ▂▂ ▅▂     ▂      ▂       
  █▁▁▁▅▅█▅▅▅██▅█▅█████████▅▅▅██▅██▅█▅███▅██████▁███▅▁▁▁█▁▁▁▁▅ ▅
  46.3 ms         Histogram: frequency by time        49.1 ms <

 Memory estimate: 208 bytes, allocs estimate: 8.

In [106]:
# store median runtime (in miliseconds)
benchmark_result["Python loop"] = median(bm.times) / 1e6

47.619125

### Python, numpy

Numpy is the high-performance scientific computing library for Python.

In [107]:
# bring in sum function from Numpy 
numpy_sum = pyimport("numpy")."sum"

PyObject <function sum at 0x34f9a2db0>

In [108]:
bm = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  166.167 μs … 269.834 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     173.625 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   176.411 μs ±   5.285 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▇█▅▃▂  ▂▅▄▃▃▂▁ ▁▂▃▂▂▂▂▂▁▁▁▂▂▂▁▁    ▁            ▂
  ▃▁▁▁▁▁▁▁▁▁▁▁▁▇███████████████████████████████████████▇▇▆▇▇▆▇▇ █
  166 μs        Histogram: log(frequency) by time        195 μs <

 Memory estimate: 208 bytes, allocs estimate: 8.

In [109]:
# store median runtime (in miliseconds)
benchmark_result["Python numpy"] = median(bm.times) / 1e6

0.173625

Numpy performance is on a par with Julia build-in sum function. Both are about 3 times faster than C, possibly because of more cache-friendly recursive mapreduce implementation.

### Julia, builtin sum

`@time`, `@elapsed`, `@allocated` macros in Julia report run times and memory allocation.

In [110]:
@time sum(x) # no compilation time after first run

  0.000170 seconds (1 allocation: 16 bytes)


500156.3977288406

For more robust benchmarking, we use BenchmarkTools.jl package.

In [111]:
bm = @benchmark sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):   92.166 μs …  4.348 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     100.209 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   103.273 μs ± 72.856 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▂     ▁▁▁▁▁  ▁█▄▁▁▃▁▂▃▂▂▁▁  ▁▁▁  ▁                          ▁
  ██▆▇▇▇▇█████▇███████████████████████████████▇▇▇▇▆▇▇▇▆▆▆▅▆▅▆▆ █
  92.2 μs       Histogram: log(frequency) by time       123 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [112]:
benchmark_result["Julia builtin"] = median(bm.times) / 1e6

0.100209

### Julia, handwritten loop

Let's also write a loop and benchmark.

In [113]:
function jl_sum(A)
    s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

bm = @benchmark jl_sum($x)

BenchmarkTools.Trial: 5959 samples with 1 evaluation per sample.
 Range (min … max):  811.625 μs …  1.192 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     828.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   838.245 μs ± 31.984 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▄▄▄▄▅▄▄▅▄▄▃▃▃▃▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁   ▁ ▁▁▁                    ▂
  ████████████████████████████████████████████▇▇▇█▇▇▇▇█▅▅▆▆▇▄▄ █
  812 μs        Histogram: log(frequency) by time       939 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [114]:
benchmark_result["Julia loop"] = median(bm.times) / 1e6

0.828

**Exercise**: annotate the loop by [`@simd`](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations-1) or `LoopVectorization.@turbo` and benchmark again. 

In [115]:
using LoopVectorization

function jl_sum_turbo(A)
    s = zero(eltype(A))
    @turbo for i in eachindex(A)
      s += A[i]
    end
    s
end

bm = @benchmark jl_sum_turbo($x)
bm

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  81.791 μs … 141.875 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     88.958 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   89.420 μs ±   5.175 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▄            ▇█▂▁▁ ▁▂▃▂▂▁    ▁▁▁                            ▁
  ██▆▇▇▅▅███▇▇▆▇███████████████████████▇█▇▆▇▇▇▆▆▆▆▆▆▅▅▅▆▅▅▄▅▅▅ █
  81.8 μs       Histogram: log(frequency) by time       110 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [116]:
benchmark_result["Julia turbo"] = median(bm.times) / 1e6

0.088958

`@tturbo` macro turns on multi-threading. Note for this to function, Julia needs to be started with multi-threading.

In [117]:
function jl_sum_tturbo(A)
    s = zero(eltype(A))
    @tturbo for i in eachindex(A)
      s += A[i]
    end
    s
end

bm = @benchmark jl_sum_tturbo($x)
bm

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  42.917 μs … 142.083 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     45.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   46.175 μs ±   4.492 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▆▅▅█▂  ▄▃▁   ▂▁▁▁▁                                        ▁
  ▄▂▅█████████████████████▇▇█████▇█▆██▇▆▇▆▅▅▅▅▆▅▄▅▅▅▄▅▅▄▅▃▄▂▅▄ █
  42.9 μs       Histogram: log(frequency) by time      62.8 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [118]:
benchmark_result["Julia tturbo"] = median(bm.times) / 1e6

0.045

### Summary

In [119]:
sort(collect(benchmark_result), by = x -> x[2])

11-element Vector{Pair{Any, Any}}:
   "Julia tturbo" => 0.045
    "Julia turbo" => 0.088958
  "Julia builtin" => 0.100209
   "Python numpy" => 0.173625
              "C" => 0.826958
     "Julia loop" => 0.828
         "R Rcpp" => 0.8309880213346332
      "R builtin" => 1.3692360371351242
         "R loop" => 9.919499047100544
 "Python builtin" => 38.529208
    "Python loop" => 47.619125

* `C`, `R builtin`, and `RCpp` are the baseline C or C++ performance (gold standard).

* `Python builtin` and `Python loop` are >50 fold slower than C because the loop is interpreted.

* `R loop` is about 10 times slower than C and indicates the performance of bytecode generated by its `compiler` package (turned on by default since R v3.4.0 (Apr 2017)). 

* `Julia loop` is close to C performance, because Julia code is JIT compiled to machine code.

* `Julia builtin` and `Python numpy` are 4-5 fold faster than C because of more cache-friendly algorithm (recursive mapreduce).

* `@turbo` (SIMD) and `@tturbo` (multithreading+SIMD), offered by the LoopVectorization.jl package. yields the top performance on this machine.

## Take home message for computational scientists

- High-level language (R, Python, Matlab) programmers should be familiar with existing high-performance packages.  Don't reinvent wheels.  
    - R: RcppEigen, tidyverse, ...   
    - Python: numpy, scipy, ...  

- In most research projects, looping is unavoidable. Then we need to use a low-level language.  
    - R: Rcpp, ...  
    - Python: Cython, ...  
    
- In this course, we use Julia, which circumvents the two language problem. So we can spend more time on algorithms, not on juggling $\ge 2$ languages.  